In [1]:
# ============================================================================================
# PROBLEM B5
#
# Build and train a neural network model using the Daily Max Temperature.csv dataset.
# Use MAE as the metrics of your neural network model.
# We provided code for normalizing the data. Please do not change the code.
# Do not use lambda layers in your model.
#
# The dataset used in this problem is downloaded from https://github.com/jbrownlee/Datasets
#
# Desired MAE < 0.2 on the normalized dataset.
# ============================================================================================

In [4]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import csv
import urllib

In [5]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

In [6]:
data_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-max-temperatures.csv'
urllib.request.urlretrieve(data_url, 'daily-max-temperatures.csv')

('daily-max-temperatures.csv', <http.client.HTTPMessage at 0x7f903fdb1df0>)

In [9]:
time_step = []
temps = []

with open('daily-max-temperatures.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    step = 0
    for row in reader:
        temps.append(row[1])
        time_step.append(step)
        step=step + 1

In [12]:
series=np.array(temps, dtype=float)

# Normalization Function. DO NOT CHANGE THIS CODE
min=np.min(series)
max=np.max(series)
series -= min
series /= max
time=np.array(time_step)

In [13]:
# DO NOT CHANGE THIS CODE
split_time=2500

time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]

In [17]:
# DO NOT CHANGE THIS CODE
window_size=64
batch_size=256
shuffle_buffer_size=1000

train_set=windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)
print(train_set)
print(x_train.shape)

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None), TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None))>
(2500,)


In [22]:
class thecustomcallbacks(tf.keras.callbacks.Callback):
    def __init__(self):
        super(thecustomcallbacks, self).__init__()
    
    def on_epoch_end(self, epoch, logs=None):
        loggedtrain = logs["mae"]
        if loggedtrain < 0.2:
            self.model.stop_training = True

In [23]:
def get_model():
    model=tf.keras.Sequential([
        tf.keras.layers.LSTM(128, return_sequences=True, input_shape=(None, 1)),
        tf.keras.layers.LSTM(64),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1, "relu")        #Regression
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.MSE, metrics = "mae")
    
    return model

In [26]:
model=get_model()
model.fit(train_set, epochs=100, callbacks=thecustomcallbacks())

Epoch 1/100
10/10 [==============================] - 4s 51ms/step - loss: 0.0583 - mae: 0.1961


2024-02-16 23:25:24.815833: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 272346020111379130
2024-02-16 23:25:24.815913: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 9801724871330745010


In [27]:
# ============================================================================================
# PROBLEM B5
#
# Build and train a neural network model using the Daily Max Temperature.csv dataset.
# Use MAE as the metrics of your neural network model.
# We provided code for normalizing the data. Please do not change the code.
# Do not use lambda layers in your model.
#
# The dataset used in this problem is downloaded from https://github.com/jbrownlee/Datasets
#
# Desired MAE < 0.2 on the normalized dataset.
# ============================================================================================

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import csv
import urllib


def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)


def solution_B5():
    data_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-max-temperatures.csv'
    urllib.request.urlretrieve(data_url, 'daily-max-temperatures.csv')

    time_step = []
    temps = []
    with open('daily-max-temperatures.csv') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        next(reader)
        step = 0
        for row in reader:
            temps.append(row[1])
            time_step.append(step)
            step=step + 1

    series= np.array(temps, dtype=float)

    # Normalization Function. DO NOT CHANGE THIS CODE
    min=np.min(series)
    max=np.max(series)
    series -= min
    series /= max
    time=np.array(time_step)
    
    #train_test_split
    split_time=2500
    time_train = time[:split_time]
    x_train = series[:split_time]
    time_valid = time[split_time:]
    x_valid = series[split_time:]
    
    #const
    window_size=64
    batch_size=256
    shuffle_buffer_size=1000

    train_set=windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)
    print(train_set)
    print(x_train.shape)

    class thecustomcallbacks(tf.keras.callbacks.Callback):
        def __init__(self):
            super(thecustomcallbacks, self).__init__()
        
        def on_epoch_end(self, epoch, logs=None):
            loggedtrain = logs["mae"]
            if loggedtrain < 0.2:
                self.model.stop_training = True

    def get_model():
        model=tf.keras.Sequential([
            tf.keras.layers.LSTM(128, return_sequences=True, input_shape=(None, 1)),
            tf.keras.layers.LSTM(64),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(1, "relu")
        ])
        model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.MSE, metrics = "mae")
        
        return model

    model=get_model()
    model.fit(train_set, epochs=100, callbacks=thecustomcallbacks())
    
    return model


# The code below is to save your model as a .h5 file.
# It will be saved automatically in your Submission folder.
if __name__ == '__main__':
    # DO NOT CHANGE THIS CODE
    model=solution_B5()
    model.save("Model/model_B5.h5")


<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None), TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None))>
(2500,)
Epoch 1/100
10/10 [==============================] - 4s 58ms/step - loss: 0.0323 - mae: 0.1389


2024-02-16 23:27:47.029373: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 272346020111379130
/root/miniconda3/envs/conda3.9/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
